### Importing Packages

In [1]:
import json
from openai import AzureOpenAI

### Defining custom functions

In the cell below, custom functions are created

In [2]:
def custom_alpha(number_list):
    # Custom function
    return (number_list[0] + number_list[1]) * (
        2 * number_list[0] + (number_list[0] * number_list[1])
    )


def custom_beta(input_string):
    # Custom function

    working_string = input_string.lower()

    # Split the string into words based on spaces
    words = working_string.split()
    # Sort the words
    sorted_words = sorted(words)
    # Reconstruct the string with spaces between sorted words
    result = " ".join(sorted_words)

    return result

### Setting up the client

In [3]:
client = AzureOpenAI(
    azure_endpoint="https://giles-openai-test.openai.azure.com/",
    api_key="27e9399d48b64ce99400c08715dcec36",
    api_version="2023-03-15-preview",
)

deployment_name = "giles-gpt-4o"

### System prompt

A system prompt is added to initialize how the model should "act"

In [4]:
system_prompt = "You are an AI assistant. You are to assist the user on their requests. You also have access to certain functions. Please utilize them when necessary."

message_list = [
    {
        "role": "system",
        "content": system_prompt,
    }
]

### Defining of tools

Here are the tools accessible by the model

In [5]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "custom_alpha",
            "description": "Perform the Custom Alpha function on a list of numbers provided. Only do this when the user is specifically asking for custom alpha to be performed",
            "parameters": {
                "type": "object",
                "properties": {
                    "numbers": {
                        "type": "array",
                        "items": {"type": "number"},
                        "description": "A list of numbers to perform the Custom Alpha function on. Example of a list is 2, 4, 8, and 16",
                    },
                },
                "required": ["numbers"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "custom_beta",
            "description": "Perform the Custom Beta function on a string. Only do this when the user is specifically asking for custom beta to be performed",
            "parameters": {
                "type": "object",
                "properties": {
                    "text": {
                        "type": "string",
                        "description": "A string to perform Custom Beta function on. Example is 'Hey I just met you'",
                    },
                },
                "required": ["text"],
            },
        },
    },
]

### User interaction

In [6]:
user_prompt = str(input("Enter your prompt: "))

print(f"\n{user_prompt}\n")

message_list.append(
    {
        "role": "user",
        "content": user_prompt,
    }
)


Custom beta the following I, I loved you in secret First sight, yeah, we love without reason Oh, twenty-five years old Oh, how were you to know, and My, my love had been frozen Deep blue, but you painted me golden Oh, and you held me close Oh, how was I to know that



### Response

The prompt is sent to the model

In [7]:
response = client.chat.completions.create(
    model=deployment_name,
    messages=message_list,
    tools=tools,
    tool_choice="auto",
)

response_message = response.choices[0].message
message_list.append(response_message)

tool_call_presence = False

if response_message.tool_calls:

    tool_call_presence = True

    for tool_call in response_message.tool_calls:

        if tool_call.function.name == "custom_alpha":

            function_args = json.loads(tool_call.function.arguments)
            print(f"Function arguments: {function_args}")

            function_response = custom_alpha(
                number_list=function_args.get("numbers", [])
            )

            print(function_response)

            message_list.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": "custom_alpha",
                    "content": str(function_response),
                }
            )

        if tool_call.function.name == "custom_beta":

            function_args = json.loads(tool_call.function.arguments)
            print(f"Function arguments: {function_args}")

            function_response = custom_beta(input_string=function_args.get("text", []))

            print(function_response)

            message_list.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": "custom_beta",
                    "content": str(function_response),
                }
            )

else:
    tool_call_presence = False

Function arguments: {'text': 'I, I loved you in secret First sight, yeah, we love without reason Oh, twenty-five years old Oh, how were you to know, and My, my love had been frozen Deep blue, but you painted me golden Oh, and you held me close Oh, how was I to know that'}
and and been blue, but close deep first frozen golden had held how how i i i, in know know, love love loved me me my my, oh, oh, oh, oh, old painted reason secret sight, that to to twenty-five was we were without yeah, years you you you you


### Final Response

If no function is called, we get the regular completion, where the model response to the prompt.

If a function is called, the result of the function must be sent back to the model so that the model can return it in a conversational manner

In [8]:
if not tool_call_presence:
    print("No Function Called")
    print(f"User prompt: {user_prompt}")
    print(f"Model response: {response_message.content}")

else:
    print("Function Called")
    response = client.chat.completions.create(
        model=deployment_name,
        messages=message_list,
    )

    print(f"User prompt: {user_prompt}")
    print(f"Model response: {response.choices[0].message.content}")


Function Called
User prompt: Custom beta the following I, I loved you in secret First sight, yeah, we love without reason Oh, twenty-five years old Oh, how were you to know, and My, my love had been frozen Deep blue, but you painted me golden Oh, and you held me close Oh, how was I to know that
Model response: Here is the custom beta of your text:

```
and and been blue, but close deep first frozen golden had held how how i i i, in know know, love love loved me me my my, oh, oh, oh, oh, old painted reason secret sight, that to to twenty-five was we were without yeah, years you you you you
```

If you have any other requests or need further modifications, feel free to let me know!


### Extra

In [9]:
for message in message_list:
    print(f"\n{message}\n")


{'role': 'system', 'content': 'You are an AI assistant. You are to assist the user on their requests. You also have access to certain functions. Please utilize them when necessary.'}


{'role': 'user', 'content': 'Custom beta the following I, I loved you in secret First sight, yeah, we love without reason Oh, twenty-five years old Oh, how were you to know, and My, my love had been frozen Deep blue, but you painted me golden Oh, and you held me close Oh, how was I to know that'}


ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_pXxkm6jCuCqyJl6PLQuBOrmD', function=Function(arguments='{"text":"I, I loved you in secret First sight, yeah, we love without reason Oh, twenty-five years old Oh, how were you to know, and My, my love had been frozen Deep blue, but you painted me golden Oh, and you held me close Oh, how was I to know that"}', name='custom_beta'), type='function')])


{'tool_call_id': 'call_